## 2019

In [68]:
import pandas as pd
import numpy as np

df2019 = pd.read_excel('./data/2019.xlsx', sheet_name= 'Sheet1', usecols=[i + 2 for i in range(5)], skiprows=1, nrows=55)
df2019.columns = ['팀 코드', '주제명', '팀원1', '팀원2', '팀원3']

df2019.head()

,팀 코드,주제명,팀원1,팀원2,팀원3
0,가-1,어떻게 해수욕장 인구를 효율적으로 측정할까?,2115 박장현,2117 송지원,NaN
1,가-2,"해수욕장, 언제 갈래요?",2401 곽서현,2402 박지윤,NaN
2,가-3,이안류 위험도를 예측하여 대비할 수 있을까?,2403 이영진,2503 박소예,NaN
3,가-4,어떻게 낚시 이용자를 증대시키고 해얀지역 경제를 활성화시킬 수 있을까?,2506 진혜민,2510 노윤호,NaN
4,가-5,해수욕장을 100%로 즐기려면?,2208 김도훈,2314 이정홍,NaN


In [69]:
df2019['팀 코드'] = df2019['팀 코드'].str.replace('가', 'A')
df2019['팀 코드'] = df2019['팀 코드'].str.replace('나', 'B')
df2019['팀 코드'] = df2019['팀 코드'].str.replace('다', 'C')
df2019['팀 코드'] = df2019['팀 코드'].str.replace('라', 'D')

df2019['팀 코드'] = df2019['팀 코드'].apply(lambda x:x.split('-')[0] + x.split('-')[1].zfill(2))

df2019['팀원1학번'], df2019['팀원1이름'] = df2019['팀원1'].apply(lambda x: x[:4] if x is not np.nan else x), df2019['팀원1'].apply(lambda x: x[4:] if x is not np.nan else x)

df2019['팀원2학번'], df2019['팀원2이름'] = df2019['팀원2'].apply(lambda x: x[:4] if x is not np.nan else x), df2019['팀원2'].apply(lambda x: x[4:] if x is not np.nan else x)

df2019['팀원3학번'], df2019['팀원3이름'] = df2019['팀원3'].apply(lambda x: x[:4] if x is not np.nan else x), df2019['팀원3'].apply(lambda x: x[4:] if x is not np.nan else x)
df2019.drop(['팀원1', '팀원2', '팀원3'], axis=1, inplace= True)

df2019.tail()

,팀 코드,주제명,팀원1학번,팀원1이름,팀원2학번,팀원2이름,팀원3학번,팀원3이름
48,D08,어선의 최적항로는 어떤 모양일까?,3408,김장현,3409,김재영,NaN,NaN
49,D09,소형 배로 적조 문제를 해결하고 해양데이터까지 수집하면 어떨까?,3416,서영국,3504,공명준,NaN,NaN
50,D10,뱃멀미를 어떻게 예방할 수 있을까?,1110,배시우,1216,이상민,1518,조성현
51,D11,해류와 암초의 영향을 고려한 선박의 최적 경로는?,1108,박정원,1109,박주영,1415,이유신
52,D12,최적의 선박 항로는? 골든 타임 확보를 위한 선박의 안전성 보장,1406,김원준,1419,최강훈,NaN,NaN


In [70]:
properties = ['팀 코드', '주제명', '팀원1 학번', '팀원1 이름', '팀원2 학번', '팀원2 이름', '팀원3 학번', '팀원3 이름']

def get_student_number(student_number):
  try:
    return str(int(student_number))
  except:
    return '0000'

def set_metadata(team_info, document, year = 2019):
  document.metadata = {}
  team_info.fillna('0')

  document.metadata['Year'] = str(year)
  document.metadata['Team code'] = str(team_info['팀 코드'])
  document.metadata['Title'] = str(team_info['주제명'])

  try:
    document.metadata['Team name'] = str(team_info['팀명'])
  except:
    document.metadata['Team name'] = '0'

  document.metadata['Teammate #1 name'] = str(team_info['팀원1이름'])
  document.metadata['Teammate #1 number'] = get_student_number(team_info['팀원1학번'])
  document.metadata['Teammate #2 name'] = str(team_info['팀원2이름'])
  document.metadata['Teammate #2 number'] = get_student_number(team_info['팀원2학번'])
  document.metadata['Teammate #3 name'] = str(team_info['팀원3이름'])
  document.metadata['Teammate #3 number'] = get_student_number(team_info['팀원3학번'])
  document.metadata['Teammate #4 name'] = 'nan'
  document.metadata['Teammate #4 number'] = 'nan'
  document.metadata['Physics'] = 'False'
  document.metadata['Chemistry'] = 'False'
  document.metadata['Biology'] = 'False'
  document.metadata['EarthScience'] = 'False'
  
  
  try:
    document.metadata['Youtube link'] = str(team_info['YOUTUBE 영상 주소'])
  except:
    document.metadata['Youtube link'] = '0'

  return document




In [71]:
import os
from langchain_community.document_loaders import PyMuPDFLoader

import re	# 정규식 라이브러리
p = re.compile('\d+') # 괄호 안에 있는 학번 추출
# 목록 제거하기
PDF_DIR = './data/2019_OceanICT/'
files = os.listdir(PDF_DIR)
files = [file for file in files if file.endswith(".pdf")]
file_list = sorted(files, key = lambda x: int(p.match(x).group()))

split_documents19 = []

for f in file_list:
  loader = PyMuPDFLoader(PDF_DIR + f)
  document = loader.load()[0]
  number = p.findall(f)[1]

  for i in range(1,4):
    result = df2019.loc[df2019[f'팀원{i}학번'] == number]
    if not result.empty:
      break

  try:
    new_document = set_metadata(result.iloc[0], document)
    split_documents19.append(new_document)
  except:
    continue

In [72]:
print(len(split_documents19))

54


In [73]:
from pprint import pprint

type(split_documents19[0])

langchain_core.documents.base.Document

## 2020

In [74]:
df20 = pd.read_excel('./data/2020.xlsx', sheet_name= '팀 코드 정리', usecols=[i + 8 for i in range(7)], skiprows=3, nrows=72)
df20.drop(['유형'], axis=1, inplace=True)
df20 = df20.rename(columns={'연구 제목': '주제명'})
df20.tail() 

,팀원1,팀원2,팀원3,주제명,팀 번호,YOUTUBE 영상 주소
66,2406 김근형,2409 김준우,NaN,효율적인 등대 배치를 위한 통계적인 최적화 방법에 대한 탐구,D-8,https://youtu.be/yXW9bSgih-Y
67,2407 김민성,2412 류운비,NaN,연안에서 최적의 선박 회피경로 구하기,D-9,https://youtu.be/7qKrCrHhFQc
68,2520 최호영,NaN,NaN,선박 간 정보공유 플랫폼 구축을 통한 어획의 편의성 증대,D-10,https://youtu.be/SB1BPdxQ-h4
69,3214 정상,3314 한경찬,NaN,선박에서 코로나바이러스 피해를 줄일 수 있을까?,D-11,https://youtu.be/sV8qrHp3vcs
70,3307 박관우,3407 박성현,NaN,선박 사고 시 가장 효율적인 대처 방안 모색,D-12,https://youtu.be/-tG4NkGq__4


In [75]:
df20['팀 코드'] = df20['팀 번호'].apply(lambda x:x.split('-')[0] + x.split('-')[1].zfill(2))

for i in range(1, 4):
    df20[f'팀원{i}학번'], df20[f'팀원{i}이름'] = df20[f'팀원{i}'].apply(lambda x: x[:4] if x is not np.nan else x), df20[f'팀원{i}'].apply(lambda x: x[5:] if x is not np.nan else x)

df20.drop(['팀 번호', '팀원1', '팀원2', '팀원3'], axis=1, inplace= True)

df20.head()

,주제명,YOUTUBE 영상 주소,팀 코드,팀원1학번,팀원1이름,팀원2학번,팀원2이름,팀원3학번,팀원3이름
0,해운대 해수욕장 파속측정을 통한 이안류 발생시 알림 수신 프로그래밍,https://www.youtube.com/watch?v=JSnm4dtmU5c,A01,1118,이현승,1115,이영진,1308,김정현
1,해양데이터를 이용해 이안류 피해를 줄일 수 있을까?,https://www.youtube.com/watch?v=Y5YvaB7awd4,A02,1210,안동준,1212,이건희,1215,장연수
2,해양 양식장 관리 시스템 설계,https://youtu.be/En4Bn1UEuyw,A03,1216,정민서,1418,장문정,1515,이승헌
3,수온에 따른 어획량예측,https://youtu.be/eNqs5N7W9v8,A04,1311,박정윤,1312,백재승,1320,정윤상
4,해수욕장 인명사고 예방을 위한 경보 시스템 제작,https://www.youtube.com/watch?v=m4bCSDfgRaA,A05,2103,안주연,2104,이수민,NaN,NaN


In [76]:
import os
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import UnstructuredPowerPointLoader


import re	# 정규식 라이브러리
p = re.compile('\d+') # 괄호 안에 있는 팀코드 추출

DIR = './data/2020_OceanICT/'
file_list = os.listdir(DIR)
file_list_pdf = [file for file in file_list if file.endswith(".pdf")]
file_list_pptx = [file for file in file_list if file.endswith(".pptx")]

print(file_list_pdf)
print(file_list_pptx)

split_documents20 = []

for f in file_list_pdf:
  loader = PyMuPDFLoader(DIR + f)
  document = loader.load()[0]
  number = p.findall(f)[1]

  for i in range(1,4):
    result = df20.loc[df20[f'팀원{i}학번'] == number]
    if not result.empty:
      break

  new_document = set_metadata(result.iloc[0], document, 2020)
  split_documents20.append(new_document)

  new_document = set_metadata(result.iloc[0], document, 2020)
  split_documents20.append(new_document)

['C8_2510양진우_2516이진혁.pdf', 'A4_1311 박정윤_1312 백재승_1320 정윤상.pdf', 'A3_ 1216 정민서 1418 장문정 1515 이승헌.pdf', 'D7_2401박가림_2408김민준.pdf', 'B21 2311 박유빈_2316 윤신재.pdf', 'D8_2406 김근형_2409 김준우.pdf', 'A1_1118이현승, 1115이영진,1308김정현.pdf', 'B18_2307김태경_2314안태영.pdf', 'B22_2315 오도영_2317 이중호 포스터 pptx.pdf', 'B8_1307 김시윤_1514 신희재_1516 이재민.pdf', 'B17_2217조원태_2218주우진.pdf', 'C2_1407 공민재_1408 권재윤_1415 성동헌.pdf', 'D3_2208김원준_2212송재훈.pdf', 'B_15 2207 김민성 2213 이찬희.pdf', 'B30_3210 성태헌_3411 송지원.pdf', 'B16_2216 정재운_2219 최유범.pdf', 'C1_1112박재한_1213이유찬_1220허준우.pdf', 'B14_2203윤지혜_2204이영은.pdf', 'B9_2101 김소연_2102 손유빈.pdf', 'B31_3508 김재진 3509 배재현.pdf', 'B25_2502장사랑_2518최우빈.pdf', 'A16_2508 송강현_2519 최준서.pdf', 'A5_2103 안주연_2104 이수민.pdf', 'A12_2313 안정욱 2321 김도준.pdf', 'B23_2402서한주_2403이성주.pdf', 'A17_3201 김소현_3304 이서현_3402 박지윤.pdf', 'B7_1309 김준영_1317 이정훈_1507 강태원.pdf', 'D9_2407김민성_2412류운비.pdf', 'B4_1119 정재민_1120 주홍성_1209 박주환.pdf', 'B15 2207김민

In [77]:
len(split_documents20)

144

## 2021

In [78]:
df21_A = pd.read_excel('./data/2021.xlsx', sheet_name= '해양 문화와 관광 진흥', usecols=[i + 2 for i in range(5)], skiprows=1, nrows=20)
df21_B = pd.read_excel('./data/2021.xlsx', sheet_name= '해양 생태계 및 환경 보존', usecols=[i + 2 for i in range(5)], skiprows=1, nrows=30)
df21_C = pd.read_excel('./data/2021.xlsx', sheet_name= '해양 자원의 이용 기반 구축', usecols=[i + 2 for i in range(5)], skiprows=2, nrows=34)
df21_D = pd.read_excel('./data/2021.xlsx', sheet_name= '해양 선박 관련 기술', usecols=[i + 2 for i in range(5)], skiprows=1, nrows=10)

df21 = pd.concat([df21_A, df21_B, df21_C, df21_D])
df21 = df21.rename(columns={'주제': '주제명'})
df21.head()

,연번,주제명,팀원1,팀원2,팀원3
0,A-1,해변 관광환경 개선 및 만족도 향상을 위한 해변 추천 서비스 제작하기,1413 신재현,1420 하승훈,NaN
1,A-2,해양 정보를 이용한 플레이리스트 추천 프로그램,2104 윤현서,2105 이다은,NaN
2,A-3,인공지능과 오픈 API를 이용한 해양 경보 어플리케이션 제작,2108 김경태,2119 최준영,NaN
3,A-4,부산 수산시장의 해산물 가격 판단 서비스,2213 안동준,2215 장연수,NaN
4,A-5,역기압 효과를 이용한 파고 높이 계산을 통해 어떻게 풍랑주의보 알리미 제작할 수 있을까?,2217 정재민,2218 조현준,NaN


In [79]:
df21['팀 코드'] = df21['연번'].apply(lambda x:x.split('-')[0] + x.split('-')[1].zfill(2))
df21['연번'] = df21['연번'].str.replace('-', '')

for i in range(1, 4):
    df21[f'팀원{i}학번'], df21[f'팀원{i}이름'] = df21[f'팀원{i}'].apply(lambda x: x[:4] if x is not np.nan else x), df21[f'팀원{i}'].apply(lambda x: x[5:] if x is not np.nan else x)

df21.drop(['팀원1', '팀원2', '팀원3'], axis=1, inplace= True)

df21.head()

,연번,주제명,팀 코드,팀원1학번,팀원1이름,팀원2학번,팀원2이름,팀원3학번,팀원3이름
0,A1,해변 관광환경 개선 및 만족도 향상을 위한 해변 추천 서비스 제작하기,A01,1413,신재현,1420,하승훈,NaN,NaN
1,A2,해양 정보를 이용한 플레이리스트 추천 프로그램,A02,2104,윤현서,2105,이다은,NaN,NaN
2,A3,인공지능과 오픈 API를 이용한 해양 경보 어플리케이션 제작,A03,2108,김경태,2119,최준영,NaN,NaN
3,A4,부산 수산시장의 해산물 가격 판단 서비스,A04,2213,안동준,2215,장연수,NaN,NaN
4,A5,역기압 효과를 이용한 파고 높이 계산을 통해 어떻게 풍랑주의보 알리미 제작할 수 있을까?,A05,2217,정재민,2218,조현준,NaN,NaN


In [80]:
import os
from langchain_community.document_loaders import PyMuPDFLoader

import re	# 정규식 라이브러리

PDF_DIR = './data/2021_OceanICT/'
file_list = os.listdir(PDF_DIR)

split_documents21 = []
a= []

for f in file_list:
  loader = PyMuPDFLoader(PDF_DIR + f)
  try:
    document = loader.load()[0]
  except:
    continue
  result = df21.loc[df21[f'연번'] == f[:-4]]

  new_document = set_metadata(result.iloc[0], document, year = 2021)
  a.append(result.iloc[0])
  split_documents21.append(new_document)

In [81]:
import pandas as pd

team_data = './data/2023.csv'

df = pd.read_csv(team_data)
df.head()

,팀 코드,주제명,해양 관련 분야 선택,분야,팀원1 학번,팀원1 이름,팀원2 학번,팀원2 이름,팀원3 학번,팀원3 이름,기타 과목,물리학,화학,생명과학,지구과학,유튜브 링크
0,A01,아두이노와 수위조절센서를 이용한 안전 부표 제작,해양 문화와 관광 진흥,A,1418,임성훈,1406.0,이서경,NaN,NaN,NaN,v,NaN,NaN,v,https://youtube.com/watch?v=8TJ7Fig9mOY&featur...
1,A02,"거리와 물때, 조수간만의 차, 해양 오염 정도를 고려한 갯벌 추천",해양 문화와 관광 진흥,A,2304,이지원,2308.0,권민찬,NaN,NaN,NaN,NaN,NaN,V,V,https://youtu.be/6CeiASdQX94
2,A03,ACO algorithm을 이용해 2030 엑스포를 위한 ​해상 도시 계획의 교통문...,해양 문화와 관광 진흥,A,2311,김민수,2312.0,김채준,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://youtu.be/5B3b9EZyLdU
3,A04,기후위기와 지역별 해적의 상관관계,해양 문화와 관광 진흥,A,2313,도영운,2314.0,신승창,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/watch?v=gAL6a0S56UM&t=3
4,A05,인공지능아~ 해운대 까페에서 먹을 음료 알려줘,해양 문화와 관광 진흥,A,2318,정민영,2319.0,정보권,NaN,NaN,V(관광 서비스),NaN,NaN,NaN,NaN,https://youtu.be/7M3h2ApRLB4


In [82]:
# 단계 1: 문서 로드(Load Documents)
# 단계 2: 문서 분할(Split Documents) -> 스킵 (사유: 포스터 섞이더라)

import os
import tqdm
from langchain_community.document_loaders import PyMuPDFLoader

import re	# 정규식 라이브러리
p = re.compile('\(([^)]+)') # 괄호 안에 있는 팀코드 추출

#PDF_DIR = './data/pdfs/2023/'
PDF_DIR = './data/2023_OceanICT/'
file_list = os.listdir(PDF_DIR)
file_list = [file for file in file_list if file.endswith(".pdf")]
split_documents23 = []

for f in tqdm.tqdm(file_list):
  loader = PyMuPDFLoader(PDF_DIR + f)
  document = loader.load()[0]

  team_code = p.findall(f)[0]

  result = df.loc[df['팀 코드'] == team_code]
  result.columns = ['팀 코드', '주제명', '1', '2', '팀원1학번', '팀원1이름', '팀원2학번', '팀원2이름', '팀원3학번', '팀원3이름', 'etc', 'Physics', 'Chemistry', 'Biology', 'EarthScience', 'YOUTUBE 영상 주소']

  subjectnames = ['Physics', 'Chemistry', 'Biology', 'EarthScience']

  subjects = []

  for i in subjectnames:
    if list(result[i])[0] in ['v', 'V']:
      subjects.append(i)


  try:
    new_document = set_metadata(result.iloc[0], document, '2023')
    for i in subjects:
      new_document.metadata[i] = 'True'
    split_documents23.append(new_document)
  except:
    continue
    #split_documents.append(new_document)

  4%|▍         | 4/89 [00:00<00:04, 18.15it/s]

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type



100%|██████████| 89/89 [00:02<00:00, 30.30it/s]


## 2024

In [83]:
df24 = pd.read_excel('./data/2024.xlsx', sheet_name= '팀별', usecols=[i + 1 for i in range(9)], skiprows=2, nrows=98)

df24 = df24.rename(columns={'섹션\n번호': '팀 코드'})
df24.drop(0, inplace=True)
df24.drop(['학년'], axis= 1,inplace=True)

df24['팀 코드'] = df24['팀 코드'].apply(lambda x:x.split('-')[0] + x.split('-')[1].zfill(2))
df24.fillna('0000', inplace=True)

for i in range(1, 4):
    try:
        df24[f'팀원{i}학번'] = df24[f'팀원{i} 학번'].astype(int).astype(str)
        df24.rename(columns={f'팀원{i} 이름': f'팀원{i}이름'},inplace=True)
        df24.drop([f'팀원{i} 학번'],inplace=True,axis=1)
    except:
        continue

df24['주제명'] = df24['팀명']
df24['과목'] = ['' for i in range(df24.shape[0])]
df24['얘 들어갔어요'] = np.zeros(df24.shape[0],dtype=int)

/var/folders/3q/p3q3k70d5919dd_gh1m0phs80000gn/T/ipykernel_77394/2635042191.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0000' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df24.fillna('0000', inplace=True)


In [84]:
df24_1 = pd.read_excel('./data/2024_1.xlsx', sheet_name = '팀별',usecols=[i + 2 for i in range(14)], skiprows=2, nrows=98)

df24_1.drop(0, inplace=True)
df24_1.drop(df24_1.columns[6:8],axis=1,inplace=True)
df24_1.columns = ['팀원1 학번','팀원1 이름', '팀원2 학번','팀원2 이름','팀원3 학번','팀원3 이름','물리','화학','생명과학','지구과학','기타', '주제명']

df24_1.head(10)

,팀원1 학번,팀원1 이름,팀원2 학번,팀원2 이름,팀원3 학번,팀원3 이름,물리,화학,생명과학,지구과학,기타,주제명
1,1102.0,방윤서,1306.0,정지민,NaN,NaN,NaN,1,NaN,NaN,NaN,난류와 ph에 의한 갯녹음화지도 만들기
2,1103.0,이서현,1104.0,이채연,NaN,NaN,1,NaN,NaN,NaN,NaN,스마트 유령어구 위치 추적 시스템
3,1105.0,최연정,1106.0,하윤아,NaN,NaN,1,1,NaN,NaN,NaN,폐수 유출 지점 탐색
4,1110.0,박관호,1112.0,박준성,NaN,NaN,1,NaN,NaN,1,NaN,이안류와 영향인자의 상관관계 분석을 통한 이안류 예측 및 대응 알고리즘
5,1113.0,송민호,1116.0,오진우,NaN,NaN,NaN,NaN,1,1,NaN,멸종위기 해양생물 개체수와 서식지역 파악하기
6,1114.0,송범준,1118.0,이정훈,NaN,NaN,NaN,NaN,1,NaN,NaN,딥러닝 기반 멸종 위기 어류 판별 및 보호
7,1117.0,이걸묵,1409.0,김기환,NaN,NaN,1,NaN,NaN,NaN,NaN,전자기유도를 이용한 파도 분석 장치 및 고효율 발전기 고안
8,1203.0,장서인,1214.0,정도현,NaN,NaN,NaN,NaN,NaN,1,NaN,위도별 해수의 온도와 해수 온도 상승 위험 정도 알림
9,1205.0,공준호,1211.0,이민찬,NaN,NaN,1,NaN,NaN,1,NaN,방사능 오염수의 확산에 따른 악영향을 최소화하기 위한 방법 찾기
10,1206.0,김민준,1213.0,장현제,NaN,NaN,1,NaN,NaN,NaN,NaN,모듈형 쌍동선의 자동 결합을 위한 코드 개발


In [85]:
subject_name = ['Physics', 'Chemistry', 'Biology', 'EarthScience']

for i in range(df24_1.shape[0]):
    subjects = list(df24_1.loc[i + 1, ['물리','화학','생명과학','지구과학']])
    subject = []
    for j in range(4):
        if subjects[j] == 1:
            subject.append(subject_name[j])

    teamname = str(int(df24_1.iloc[i, 0]))
    projectname = df24_1.iloc[i, 11]

    
    df24.loc[df24[f'팀원1학번'] == teamname, '주제명'] = projectname
    df24.loc[df24[f'팀원1학번'] == teamname, '과목'] = ','.join(subject)
    for j in range(3):
        print(df24_1.iloc[i, 2*j + 1])
        df24.loc[df24[f'팀원1학번'] == teamname, f'팀원{j + 1}이름'] = df24_1.iloc[i, 2*j + 1]

df24.head(10)

방윤서
정지민
nan
이서현
이채연
nan
최연정
하윤아
nan
박관호
박준성
nan
송민호
오진우
nan
송범준
이정훈
nan
이걸묵
김기환
nan
장서인
정도현
nan
공준호
이민찬
nan
김민준
장현제
nan
신승환
정현철
nan
양승우
최문기
nan
정재현
정휘찬
nan
봉가현
정세연
nan
정은수
정지민
nan
정은수
김건영
nan
권오준
이지우
nan
김창욱
허윤
nan
노현우
최승기
nan
박예일
이재원
nan
권나경
김수현
nan
김미소
이희원
nan
장소형
장윤
nan
강준호
이찬서
nan
구은현
성민찬
nan
김건영
김정원
nan
김도훈
최신호
nan
최선웅
한서준
nan
강혜진
이인규
nan
김소원
박지유
nan
권민근
서정안
nan
김건우
박상진
정수환
김범수
황우진
nan
선우무경
안상민
nan
김지우
이재영
nan
배시아
이윤승
nan
정여진
이재호
nan
고민재
김성민
nan
김상우
박재성
nan
박준영
박찬휘
백창훈
안병훈
이승후
nan
이건우
백종윤
nan
이성민
장동주
nan
강지윤
여유진
nan
김서경
이나원
nan
전예원
박주형
nan
김동건
임건택
nan
김바다
정지훈
nan
김지홍
양성수
nan
류장현
표정훈
nan
박준서
이원혁
nan
이동윤
최현민
nan
구승현
김민서
nan
김아윤
박지환
nan
정유진
하은결
nan
김민기
김환
nan
김석현
박시형
nan
김성윤
유민석
nan
김태규
박예준
nan
이승준
김은우
nan
이언석
정우진
홍진수
김유빈
맹형주
nan
서해원
손지민
nan
은채현
남희수
nan
한예송
류홍재
nan
김도현
김찬영
송준환
김태윤
이석준
nan
안승준
허석현
nan
유예준
임성훈
nan
정우성
조은우
nan
김민채
김윤정
nan
양수빈
유서윤
nan
이서경
강석현
nan
권민성
이우진
nan
권해정
김성민
nan
김민규
김준민
nan
김태희
이호준
nan
서재원
정희도
nan
조용석
하서준
nan
권지민
김민수
이지원
김정현
조민경
nan
윤성빈
이경록
nan
김채준
노현수
nan
신동원
이윤

,팀 코드,팀원1이름,팀원2이름,팀원3이름,팀명,팀원1학번,팀원2학번,팀원3학번,주제명,과목,얘 들어갔어요
1,B01,방윤서,정지민,NaN,클로렐라쌀밥,1102,1306,0,난류와 ph에 의한 갯녹음화지도 만들기,Chemistry,0
2,B02,이서현,이채연,NaN,유령인간,1103,1104,0,스마트 유령어구 위치 추적 시스템,Physics,0
3,B03,최연정,하윤아,NaN,디버깅 마스터,1105,1106,0,폐수 유출 지점 탐색,"Physics,Chemistry",0
4,A01,박관호,박준성,NaN,정보통신기술,1110,1112,0,이안류와 영향인자의 상관관계 분석을 통한 이안류 예측 및 대응 알고리즘,"Physics,EarthScience",0
5,B04,0000,0000,0000,별미,1111,1505,0,별미,,0
6,B05,송민호,오진우,NaN,오진우뭐먹어나도줘,1113,1116,0,멸종위기 해양생물 개체수와 서식지역 파악하기,"Biology,EarthScience",0
7,B06,송범준,이정훈,NaN,바다의 왕자,1114,1118,0,딥러닝 기반 멸종 위기 어류 판별 및 보호,Biology,0
8,D01,이걸묵,김기환,NaN,바다와 고딩,1117,1409,0,전자기유도를 이용한 파도 분석 장치 및 고효율 발전기 고안,Physics,0
9,B07,장서인,정도현,NaN,SeaPerature,1203,1214,0,위도별 해수의 온도와 해수 온도 상승 위험 정도 알림,EarthScience,0
10,B08,공준호,이민찬,NaN,Radioactive BSS,1205,1211,0,방사능 오염수의 확산에 따른 악영향을 최소화하기 위한 방법 찾기,"Physics,EarthScience",0


In [86]:
df24_2 = pd.read_excel('./data/2024_2.xlsx', sheet_name = '팀별',usecols=[i + 2 for i in range(14)], skiprows=3, nrows=96)

# #df24 = df24.rename(columns={'섹션\n번호': '팀 코드'})
df24_2.drop(df24_2.columns[1:5],axis=1,inplace=True)
df24_2.drop(df24_2.columns[2:],axis=1,inplace=True)
print(df24_2.head())
for i in range(df24_2.shape[0]):
    num = str(int(df24_2.iloc[i, 1]))
    youtubelink = df24_2.iloc[i, 0]
    df24.loc[df24[f'팀원1학번'] == num, 'YOUTUBE 영상 주소'] = youtubelink
    if df24.loc[df24[f'팀원1학번'] == num].empty:
        df24.loc[df24[f'팀원2학번'] == num, 'YOUTUBE 영상 주소'] = youtubelink

                          유투브주소  Unnamed: 7
0                           NaN        1112
1  https://youtu.be/I1YT5vt_4uQ        1218
2                           NaN        1313
3                           NaN        1403
4                           NaN        3412


/var/folders/3q/p3q3k70d5919dd_gh1m0phs80000gn/T/ipykernel_77394/3780249740.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'https://youtu.be/I1YT5vt_4uQ' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df24.loc[df24[f'팀원1학번'] == num, 'YOUTUBE 영상 주소'] = youtubelink


In [87]:
df24

,팀 코드,팀원1이름,팀원2이름,팀원3이름,팀명,팀원1학번,팀원2학번,팀원3학번,주제명,과목,얘 들어갔어요,YOUTUBE 영상 주소
1,B01,방윤서,정지민,NaN,클로렐라쌀밥,1102,1306,0,난류와 ph에 의한 갯녹음화지도 만들기,Chemistry,0,https://youtu.be/q3JQTU6YkdI
2,B02,이서현,이채연,NaN,유령인간,1103,1104,0,스마트 유령어구 위치 추적 시스템,Physics,0,https://youtu.be/uaZ4pZvrCY4?si=xlo6fLN46-4BZibJ
3,B03,최연정,하윤아,NaN,디버깅 마스터,1105,1106,0,폐수 유출 지점 탐색,"Physics,Chemistry",0,https://youtu.be/GomuZLXbsNU?si=bwABobaCQKIr6KnL
4,A01,박관호,박준성,NaN,정보통신기술,1110,1112,0,이안류와 영향인자의 상관관계 분석을 통한 이안류 예측 및 대응 알고리즘,"Physics,EarthScience",0,NaN
5,B04,0000,0000,0000,별미,1111,1505,0,별미,,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
93,B53,남지연,유지한,NaN,유지한남지연,3402,3410,0,"회귀분석과 딥러닝을 이용한 기후 변화-해안 해파리 출몰의 상관관계 파악, 예측 및 ...",EarthScience,0,NaN
94,B54,강재원,김대현,NaN,장발장발장,3405,3406,0,해류 유속 분석을 통한 쓰레기 대륙 형성 경로 분석,"Physics,EarthScience",0,NaN
95,B55,김서진,이영서,NaN,영서진 짜예쁘다,3502,3511,0,"방사능 해양 안전 모니터링, 정보 제공 프로그램","Physics,Chemistry",0,NaN
96,B56,김지인,김민준,NaN,가위바위보,3503,3508,0,해앙 미세플라스틱 분해 효소의 작용 모델링,Biology,0,NaN


In [88]:
import os
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import UnstructuredPowerPointLoader
import tqdm

import re	# 정규식 라이브러리
p = re.compile('[A-D]\d{2}') # 괄호 안에 있는 팀코드 추출
p1 = re.compile('[가-힣]+')
num = re.compile('\d{4} ?[가-힣]{2,4}')

DIR = './data/2024_OceanICT/'
file_list = os.listdir(DIR)
split_documents24 = []

for f in file_list:
    if f.endswith('.pdf'):
        loader = PyMuPDFLoader(DIR + f)
    else:
        continue

    number = p.findall(f)
    document = loader.load()[0]

    result = df24.loc[df24[f'팀 코드'] == number[0]]
    if not result.empty:
        df24.loc[df24[f'팀 코드'] == number[0], '얘 들어갔어요'] = 1
    else:
        print(result)

    new_document = set_metadata(result.iloc[0], document, year = 2024)
    for j in subject_name:
        new_document.metadata[j] = 'True'
    split_documents24.append(new_document)

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type



In [89]:
df24.iloc[88]

팀 코드                                    A12
팀원1이름                                   김열린
팀원2이름                                   정민영
팀원3이름                                   NaN
팀명               SEY(Search Extreme & Yell)
팀원1학번                                  3306
팀원2학번                                  3412
팀원3학번                                     0
주제명                        든든한 나의 보디가드, 오션이
과목                          Physics,Biology
얘 들어갔어요                                   0
YOUTUBE 영상 주소                           NaN
Name: 89, dtype: object

In [90]:
df24

,팀 코드,팀원1이름,팀원2이름,팀원3이름,팀명,팀원1학번,팀원2학번,팀원3학번,주제명,과목,얘 들어갔어요,YOUTUBE 영상 주소
1,B01,방윤서,정지민,NaN,클로렐라쌀밥,1102,1306,0,난류와 ph에 의한 갯녹음화지도 만들기,Chemistry,1,https://youtu.be/q3JQTU6YkdI
2,B02,이서현,이채연,NaN,유령인간,1103,1104,0,스마트 유령어구 위치 추적 시스템,Physics,1,https://youtu.be/uaZ4pZvrCY4?si=xlo6fLN46-4BZibJ
3,B03,최연정,하윤아,NaN,디버깅 마스터,1105,1106,0,폐수 유출 지점 탐색,"Physics,Chemistry",0,https://youtu.be/GomuZLXbsNU?si=bwABobaCQKIr6KnL
4,A01,박관호,박준성,NaN,정보통신기술,1110,1112,0,이안류와 영향인자의 상관관계 분석을 통한 이안류 예측 및 대응 알고리즘,"Physics,EarthScience",1,NaN
5,B04,0000,0000,0000,별미,1111,1505,0,별미,,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
93,B53,남지연,유지한,NaN,유지한남지연,3402,3410,0,"회귀분석과 딥러닝을 이용한 기후 변화-해안 해파리 출몰의 상관관계 파악, 예측 및 ...",EarthScience,1,NaN
94,B54,강재원,김대현,NaN,장발장발장,3405,3406,0,해류 유속 분석을 통한 쓰레기 대륙 형성 경로 분석,"Physics,EarthScience",1,NaN
95,B55,김서진,이영서,NaN,영서진 짜예쁘다,3502,3511,0,"방사능 해양 안전 모니터링, 정보 제공 프로그램","Physics,Chemistry",1,NaN
96,B56,김지인,김민준,NaN,가위바위보,3503,3508,0,해앙 미세플라스틱 분해 효소의 작용 모델링,Biology,1,NaN


In [91]:
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from kkey import get_key

gemini_api_key = get_key()

# Gemini 임베딩 객체 생성
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=gemini_api_key
)

In [92]:
import pprint 

for i in split_documents24:
    print(i.metadata)

{'Year': '2024', 'Team code': 'B02', 'Title': '스마트 유령어구 위치 추적 시스템', 'Team name': '유령인간', 'Teammate #1 name': '이서현', 'Teammate #1 number': '1103', 'Teammate #2 name': '이채연', 'Teammate #2 number': '1104', 'Teammate #3 name': 'nan', 'Teammate #3 number': '0', 'Teammate #4 name': 'nan', 'Teammate #4 number': 'nan', 'Physics': 'True', 'Chemistry': 'True', 'Biology': 'True', 'EarthScience': 'True', 'Youtube link': 'https://youtu.be/uaZ4pZvrCY4?si=xlo6fLN46-4BZibJ'}
{'Year': '2024', 'Team code': 'B01', 'Title': '난류와 ph에 의한 갯녹음화지도 만들기', 'Team name': '클로렐라쌀밥', 'Teammate #1 name': '방윤서', 'Teammate #1 number': '1102', 'Teammate #2 name': '정지민', 'Teammate #2 number': '1306', 'Teammate #3 name': 'nan', 'Teammate #3 number': '0', 'Teammate #4 name': 'nan', 'Teammate #4 number': 'nan', 'Physics': 'True', 'Chemistry': 'True', 'Biology': 'True', 'EarthScience': 'True', 'Youtube link': 'https://youtu.be/q3JQTU6YkdI'}
{'Year': '2024', 'Team code': 'C10', 'Title': '미규모 와류 예측을 이용한 스마트 해양 선박 시스템 제작', 'Team 

In [93]:
split_documents23

[Document(metadata={'Year': '2023', 'Team code': 'B42', 'Title': '옥시벤존 농도 모니터링 프로그램을 통한 해양생태계 보호', 'Team name': '0', 'Teammate #1 name': '조수현', 'Teammate #1 number': '3203', 'Teammate #2 name': '홍수민', 'Teammate #2 number': '3306', 'Teammate #3 name': 'nan', 'Teammate #3 number': '0000', 'Teammate #4 name': 'nan', 'Teammate #4 number': 'nan', 'Physics': 'False', 'Chemistry': 'True', 'Biology': 'True', 'EarthScience': 'False', 'Youtube link': 'https://youtu.be/LX37iXYXHTA'}, page_content='Busan science high school\n2023 Ocean ICT Festival\n2023 BOIF\nYoutube 영상 QR\nB \n42\nQR 코드 영역\nQR 삽입 후 \n테두리 삭제\n옥시벤존농도모니터링프로그램을\n통한해양생태계보호\n조수밤바다홍수경보\n3203 조수현 3306 홍수민\n1. 탐구 목적\n[2022 Ocean ICT]\n해수의 옥시벤존 농도를 측정해 특정 값 이상이 되면 경고를 울리고 옥시벤존 분해 물질을 투여하는 코드를 작성하여 옥시벤존의 농도를 줄이는 탐구를 진행함.\n[2023 Ocean ICT]\n하지만, 해수의 옥시벤존을 직접적으로 분해시키는 방법은 매우 일시적인 효과를 가지며, 가성비와 효율이 좋지 못하다는 단점이 있었다.\n따라서, 이번에는 접근법에 변화를 주어 바다로 가는 사람들이나 해변에서 활동하는 사람들을 대상으로 옥시벤존과 옥티노세이트가 함유된 선크림을 사용하지\n않도록 경고하거나 대체 제품을 추천하는 프로그램을 제작하고자 하였다.\n[해양 

In [94]:
a = []
for i in split_documents24:
    a.append(i.metadata['Team code'])

In [95]:
print(sorted(a))

['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'B01', 'B02', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B10', 'B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B17', 'B18', 'B19', 'B20', 'B21', 'B22', 'B23', 'B24', 'B25', 'B26', 'B27', 'B28', 'B29', 'B30', 'B31', 'B32', 'B33', 'B34', 'B35', 'B36', 'B37', 'B38', 'B39', 'B40', 'B41', 'B42', 'B43', 'B44', 'B45', 'B46', 'B47', 'B48', 'B49', 'B50', 'B51', 'B52', 'B53', 'B54', 'B55', 'B56', 'C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08', 'C09', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'D09', 'D10', 'D11', 'D12', 'D13', 'D14']


In [96]:
split_documents24

[Document(metadata={'Year': '2024', 'Team code': 'B02', 'Title': '스마트 유령어구 위치 추적 시스템', 'Team name': '유령인간', 'Teammate #1 name': '이서현', 'Teammate #1 number': '1103', 'Teammate #2 name': '이채연', 'Teammate #2 number': '1104', 'Teammate #3 name': 'nan', 'Teammate #3 number': '0', 'Teammate #4 name': 'nan', 'Teammate #4 number': 'nan', 'Physics': 'True', 'Chemistry': 'True', 'Biology': 'True', 'EarthScience': 'True', 'Youtube link': 'https://youtu.be/uaZ4pZvrCY4?si=xlo6fLN46-4BZibJ'}, page_content='Busan science high school\n2024 Ocean ICT Festival\n2024 BOIF\nYoutube 영상 \nQR\nB\n02\n유령인간\n1103 \n \n이서현1104 이채연\n \n \n \n \n \n스마트유령어구위치추적시스템고안\n탐구동기\n \n융합분야( 물리, 정보과학)\n1. \n \n \n \n \n평균유속이부정확할수있다.\n-\n \n \n계절별유속, \n \n \n \n \n \n \n \n \n \n간조혹은만조때의유속은평균적인값에서큰오차를발생시킨다.\n2. \n \n \n \n \n \n \n \n \n유령어구의유실이자연적상태외다르게발생할수있다.\n-\n \n장비결함, \n \n날씨영향, \n \n선박충돌, \n \n \n \n \n기물파손혹은절도등\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n단순히어구의유실범위를추정하는것을넘어서다른기관과협력해실제로수거하는과정까지확장\n \n \n시킬수있다.\n-\n \n 

In [97]:
documents1 = []
documents2 = []
for i in ['19', '20', '21', '23', '24']:
    documents1 += globals()['split_documents' + i]



In [98]:
for idx, doc in enumerate(documents1):
    for k in doc.metadata.keys():
        if not isinstance(k, str):
            print(f"문제 발생: index={idx}, key={k}, type={type(k)}, metadata={doc.metadata}")

In [100]:
vectorstore1 = Chroma.from_documents(
    documents=documents1,
    embedding=embeddings,
    persist_directory='./gem_oldfiles'
)